### Quora preprocessing 

**(~1:30h gpu run time)**

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

### Load data

In [2]:
df_raw = pd.read_csv("../data/raw/train.csv", low_memory=False)
df_raw.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [3]:
df_raw.shape

(1306122, 3)

In [4]:
sampled_df = df_raw[df_raw.target == 0].sample(n=480_000, random_state=42)

df = pd.concat(
    [sampled_df, df_raw[df_raw.target == 1]]
)
df.head()

,qid,question_text,target
1022714,c86ab618f85e9b7fc374,Is sadism a coping mechanism for people who ar...,0
641364,7d9ea6d66b8866e69240,Is it possible for me as a soul to go outside ...,0
1225111,f01982d0cd06aba308ed,Do Pet Animal Rescue workers minimize shows of...,0
1130433,dd8a6b5452a407cea2ac,How do you identify a sonnet and what can we i...,0
1220402,ef30e73bf0a81a06ccf6,Is there a special place in hell for the likes...,0


In [5]:
# Target samples
df.shape[0] - 200_000

360810

### Preprocessing

In [6]:
# Train/Test
X, y = df.drop('target', axis=1), df.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42
)

train_df = X_train
train_df['target'] = y_train

test_df = X_test
test_df['target'] = y_test

### Random Oversampling

In [7]:
# Oversampling
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

resampled_df = X_resampled
resampled_df['target'] = y_resampled

### nlpaug

Using:
  * KeyboardAug
  * ContextualWordEmbsAug
  * SynonymAug
  * BackTranslationAug
  * SpellingAug

80_000 new examples for each augment 

In [8]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas

df_1 = train_df[train_df.target==1].copy()

texts = list(df_1.question_text)
texts[:5]

['Do you see a time where white people will become Trump supporters and start systematically killing Hispanics just like Germany did with Jewish people and other minorities back in the 1930s?',
 'Are Brahmins responsible for the perpetuation of caste system and caste-based discrimination in India?',
 'Why do the Kardashians and Jenner sisters only date black men? Why do the Kardashian sisters seem to prefer black boyfriends?',
 'Why are books about ‘how to rape a woman’ in schools in the US?',
 'How can I train my girlfriend to stop speaking out of place in public?']

In [11]:
key_board_aug = nac.KeyboardAug(aug_char_max=2, aug_word_max=2)
key_board_texts = key_board_aug.augment(texts)
key_board_texts[:5]

['Do you see a time Etere white people will become Trump supporters and start systematically killing Hispanics just like Germany did with Jewish people and other minorities vaSk in the 1930s?',
 'Are Brahmins responsible for the perpetuation of FastW system and caste - baAsd discrimination in India?',
 'Why do the KafdashiSns and Jenner sisters only date black men? Why do the Kardashian sisters seem to prefer bpacI boyfriends?',
 'Why are goiks about ‘ how to rape a w9jan ’ in schools in the US?',
 'How can I tFaLn my girlfriend to stop speaking out of Llacw in public?']

In [12]:
synonym_aug = naw.SynonymAug(aug_max=2)
synonym_texts = synonym_aug.augment(texts)
synonym_texts[:5]

['Do you see a time where white people will become Trump supporters and start systematically killing Hispanics just like Frg did with Jewish people and other minority back in the 1930s?',
 'Are Brahman responsible for the perpetuation of caste system and caste - based discrimination in India?',
 'Why do the Kardashians and Jenner sisters only date black men? Why do the Kardashian sisters seem to choose bootleg boyfriends?',
 'Why are books about ‘ how to outrage a woman ’ in schools in the u?',
 'How can I train my girlfriend to end speaking out of place in world?']

In [13]:
spelling_aug = naw.SpellingAug(aug_max=1)
spelling_texts = spelling_aug.augment(texts)
spelling_texts[:5]

['Do you see a time where white people will become Trump supporters and start systematically killing Hispanics just like Germany did with Jewish people and other minorities block in the 1930s?',
 'Are Brahmins responsible for the perpetuation or caste system and caste - based discrimination in India?',
 'Why do the Kardashians and Jenner sisters only date black men? Why do the Kardashian sisters seem to prefer blacky boyfriends?',
 'Why are books about ‘ how to rape aa woman ’ in schools in the US?',
 'How can I train my girlfriend to stop speaking out of place in bubic?']

In [9]:
# Load contextual words
contextual_words_aug = naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased', aug_max=4, device='cuda')
contextual_words_aug

In [10]:
contextual_words = []

for i in range(0, len(texts), 64):
    if i%1024==0:
        print(f"{i}/{len(texts)}")
    contextual_words += contextual_words_aug.augment(texts[i: i + 64])

contextual_words[:5]

0/72829
1024/72829
2048/72829
3072/72829
4096/72829
5120/72829
6144/72829
7168/72829
8192/72829
9216/72829
10240/72829
11264/72829
12288/72829
13312/72829
14336/72829
15360/72829
16384/72829
17408/72829
18432/72829
19456/72829
20480/72829
21504/72829
22528/72829
23552/72829
24576/72829
25600/72829
26624/72829
27648/72829
28672/72829
29696/72829
30720/72829
31744/72829
32768/72829
33792/72829
34816/72829
35840/72829
36864/72829
37888/72829
38912/72829
39936/72829
40960/72829
41984/72829
43008/72829
44032/72829
45056/72829
46080/72829
47104/72829
48128/72829
49152/72829
50176/72829
51200/72829
52224/72829
53248/72829
54272/72829
55296/72829
56320/72829
57344/72829
58368/72829
59392/72829
60416/72829
61440/72829
62464/72829
63488/72829
64512/72829
65536/72829
66560/72829
67584/72829
68608/72829
69632/72829
70656/72829
71680/72829
72704/72829


['do you see a reality where white people invariably become trump supporters and start systematically killing hispanics just like germany did with jewish people and black minorities born in the 1930s?',
 'are brahmins targeted for the discrimination upon caste system and family - based discrimination in india?',
 'where do these kardashians and similar sisters only date black men? why do the kardashian sisters seem to prefer single boyfriends?',
 'why are minds about learning how to rape a lady ’ in schools in our us?',
 'is can hell train my girlfriend to go flirting out of place in public?']

In [15]:
new_texts = key_board_texts + synonym_texts + contextual_words + spelling_texts
new_texts[:5]

['Do you see a time Etere white people will become Trump supporters and start systematically killing Hispanics just like Germany did with Jewish people and other minorities vaSk in the 1930s?',
 'Are Brahmins responsible for the perpetuation of FastW system and caste - baAsd discrimination in India?',
 'Why do the KafdashiSns and Jenner sisters only date black men? Why do the Kardashian sisters seem to prefer bpacI boyfriends?',
 'Why are goiks about ‘ how to rape a w9jan ’ in schools in the US?',
 'How can I tFaLn my girlfriend to stop speaking out of Llacw in public?']

In [16]:
df_nlpaug = pd.DataFrame({'question_text': new_texts, 'target': np.ones(len(new_texts))})
df_nlpaug.head()

,question_text,target
0,Do you see a time Etere white people will beco...,1.0
1,Are Brahmins responsible for the perpetuation ...,1.0
2,Why do the KafdashiSns and Jenner sisters only...,1.0
3,Why are goiks about ‘ how to rape a w9jan ’ in...,1.0
4,How can I tFaLn my girlfriend to stop speaking...,1.0


In [17]:
train_nlpaug = pd.concat([train_df, df_nlpaug])
train_nlpaug.head()

,qid,question_text,target
90150,11aaa9f9ddea69b5809f,What is the meaning of merits of the case?,0.0
58642,0b816d1f1762919ca21d,How do I find out the original owner of house ...,0.0
350652,44b93de11db2266b17ac,Why was Veer Savarkar linked to Mahatma Gandhi...,0.0
982543,c07df1117a2439500d86,What is the best moveset for Xurkitree?,0.0
743026,918731b2ade71a2cd655,Which is the best portal to buy dermatologists...,0.0


In [18]:
train_nlpaug.shape

(796045, 3)

In [1]:
train_nlpaug.target.value_counts()

NameError: name 'train_nlpaug' is not defined

In [2]:
import pandas as pd
t = pd.read_csv("../data/nlpaug/train.csv")
t.shape

/home/fdem/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


(796045, 4)

In [6]:
x1 = t[t.target==1]
x2 = t[t.target==0]
x2 = x2.sample(n=364145)

df = pd.concat([x1, x2])
df.shape

(728290, 4)

In [8]:
df.target.value_counts()

0.0    364145
1.0    364145
Name: target, dtype: int64

### Save

In [20]:
# train datasets
resampled_df.to_csv("../data/ros/train.csv")
train_nlpaug.to_csv("../data/nlpaug/train.csv")

# nlpaug
test_df.to_csv("../data/processed/test.csv")